<a href="https://colab.research.google.com/github/baoloc97/Data-science/blob/main/Image_Classfication_BanProductV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip ban..zip -d 'train/ban/'

In [ ]:
import numpy as np 
import cv2
import os
import random
import matplotlib.pyplot as plt 
import pickle


In [ ]:
DIRECTORY = r'train'
CATEGORIES =  ['ban','unban']


In [ ]:
IMG_SIZE = 224
data = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        new_arr = cv2.resize(arr, (60, 60))
        data.append([new_arr, label])

In [ ]:
random.shuffle(data)

In [ ]:
X = []
y = []
for features, labels in data:
	X.append(features)
	y.append(labels)

X = np.array(X)
y = np.array(y)

print(X[0])

[[180 173 185 ... 246 246 246]
 [193 173 189 ... 245 243 243]
 [197 182 194 ... 246 247 244]
 ...
 [188 196 206 ... 207 248 225]
 [244 225 231 ... 240 244 206]
 [250 249 249 ... 247 215 209]]


In [ ]:
pickle.dump(X,open('X.pkl','wb'))
pickle.dump(y,open('y.pkl','wb'))

In [ ]:
import pickle
import tensorflow
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense
from tensorflow.keras.callbacks import TensorBoard
import keras.losses
keras.losses.custom_loss = 'spare_categorical_crossentrop'
X = pickle.load(open('X.pkl','rb'))
y = pickle.load(open('y.pkl','rb'))



In [ ]:
X = X/255


In [ ]:
#y = np.asarray(y).astype('float32').reshape((1,-1))

In [ ]:
print(X.shape)
print(y.shape)

(2001, 60, 60, 1)
(2001,)


In [ ]:
X = X.reshape(-1, 60, 60, 1)

In [ ]:
#Ở đây Dùng CNN ~ 64%, google net, effecient net work better ~ 76%
model = Sequential()

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D((2,2)))

model.add(Flatten())

model.add(Dense(128, input_shape = X.shape[1:], activation = 'relu'))

model.add(Dense(2, activation = 'softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=1, validation_split=0.2,batch_size=32)

In [ ]:
import time

time.time()

1623828917.5039914

In [ ]:
model.save('3x3x64-banvsunban.model')

In [ ]:
!unzip bantest.zip -d 'test/ban/'

In [ ]:
!unzip unbantest.zip -d 'test/unban/'

In [ ]:
# mình làm predict ở đây nha
import cv2
import keras
import numpy as np
from keras.preprocessing import image
from os import listdir
from os.path import isfile, join


CATEGORIES = ['ban', 'unban']


def image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    new_arr = cv2.resize(img, (60, 60))
    new_arr = np.array(new_arr)
    new_arr = new_arr.reshape(-1, 60, 60, 1)
    return new_arr

model = keras.models.load_model('3x3x64-banvsunban.model')



In [ ]:

prediction = model.predict([image('test/predict_small/ban2.jpg')])
print(CATEGORIES[prediction.argmax()])

In [ ]:
DIRECTORY_Test = r'test'
CATEGORIES_Test =  ['ban','unban']

In [ ]:
data_test = []
for category in CATEGORIES_Test:
    path = os.path.join(DIRECTORY_Test, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES_Test.index(category)

        data_test.append([img,label])
random.shuffle(data_test)       
print(data_test[:3])  
print(len(data_test))
for i in range(len(data_test)-1000):
  print(data_test[i][1])      

In [ ]:
!unzip bantest.zip -d 'test/predict/'

In [ ]:
!unzip unbantest.zip -d 'test/predict/'

In [ ]:
data_test_size = len(data_test) 
#data_test_size = data_test_size - 980
acc =0 
mypath = "test/predict/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
for i in range(len(data_test)):
  link = join(mypath,data_test[i][0])
  prediction = model.predict([image(link)])
  if prediction.argmax() == data_test[i][1]:
    acc = acc+1
    #print(prediction.argmax())
    #print(data_test[i][1])
  #print(CATEGORIES_Test[prediction.argmax()])

print("Predict Accuracy = ", 100*float(acc/data_test_size))  

True